# Borneo Endangered Species Classifier

This is the project I did for the fast.ai lesson 2 homework. This classifer will correct classify 3 critically endangered mammals native to the Borneo Rainforest: the Borneo Orangutan, the Eastern Sumatran Rhinoceros, and the Borneo pygmy elephant. Go ahead and give it a try! If you're interested about my process and the current limitations of this model, see below.


In [2]:
from utils import *
from fastai2.vision.widgets import *

In [3]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [4]:
learn_inf = load_learner(path/'export.pkl')

In [5]:
learn_inf.predict('images/orangutan.jpg')

('borneo orangutan',
 tensor(0),
 tensor([9.9999e-01, 1.1462e-06, 4.6091e-06, 9.3500e-08]))

In [6]:
learn_inf.dls.vocab

(#4) ['borneo orangutan','borneo pygmy elephant','borneo rainforest','eastern sumatran rhinoceros']

In [7]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [8]:
img = PILImage.create(btn_upload.data[-1])

IndexError: list index out of range

In [49]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

Output()

In [50]:
pred,pred_idx,probs = learn_inf.predict(img)

In [51]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: eastern sumatran rhinoceros; Probability: 0.9996')

In [52]:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [53]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [54]:
VBox([widgets.Label('Upload a photo to see if it contains an endangered animal on Borneo!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

### Turning Your Notebook into a Real App

In [ ]:
#hide
!pip install voila
!jupyter serverextension enable voila —sys-prefix

## Data

The data consists of the three types of endangered large mammals we want to classify: the Borneo Orangutan, the Eastern Sumatran Rhinoceros, and the Borneo Pygmy Elephant. The fourth class, the borneo rainforest, is there as a sort of control. If this was used in production, presumably in photos that do not contain these animals a camera would be seeing the borneo rainforest. This is more a proof of concept than a production ready applications, the problems with the data will be discussed below.

Note: an "other animals" category may be useful for preventing misclassification of other large mammals that live in the borneo rainforest as any of the endangered species

## Conclusion and Further Research
This model works exceptionally well (>99.9% accuracy), however, there are still obstacles to practical application and  additions/alternatives that may be better suited to this task. 

### Data issues and Further Research
The data itself has some issues. All images were taken from Bing Image Search and as such aren't representive of the low resolution, differently lit photos that you would get from say placing raspberry pi's with cameras around the jungle. Most photos are stylised photos taken by professional photographers and many don't show the animals as they would appear in the jungle. Another issue is that they don't include many, if any, night time photos of the animals, which would mean that it would not identify the animals at night, however, this could be compensated by either building a dataset of these types of images or relying on other models, such as audio or heat based models at night. 

There are also problems related to the class composition of the dataset. Firstly, while it contains a control class of photos of the borneo rainforest, as mentioned above, that are likely unrepresentative of photos the cameras would take. Beyond this, a further control would be needed of "other animals", which would provide a class of many other animals in the borneo rainforest so that the model would not misclassify them as one of the three animals. Of course, you could add all of these animals as separate classes, but this would make the model more complex and likely distract from its ability to correctly identify those mammals that are most at risk. Secondly, further data point might be wise to add, that of poachers/loggers. As one of the main threats to these animals is both hunting and destruction of their environment by loggers, using the model to also track them would likely be incredibly helpful in keeping track of and alerting authorities of their presence

Beyond these there are a number of additions/substitutions to the image based model that could augment it overall as well as provide alternatives to it in situations where it may not perform well. 
- Video-based model: adding a video based model so that when the image classifer detects one of the endangered animals it could switch to a live video feed. Alternatively, you could just stick to the image based model, that increases the number of images taken once an endangered animal is detected and sends notifications to local conservationists so they can track their location and behaviour or tag untagged animals
- Audio-based model: adding a audio based model to complement or replace image based model. Build a dataset of the sounds of each animal (calls, movement sounds, etc.), convert the audio into images, use a CNN to classify them or use a RNN to just directly classify the sound. I'm not sure about the power costs of this
- Heat-based model: adding a heat based model that uses an infrared camera to classify animals. Would like be helpful during night, however, there are many issues I'm unsure of
